In [1]:
import numpy as np

class LogisticRegression(object):
  '''
  ロジスティック回帰
  '''
  def __init__(self, input_dim):
    self.input_dim = input_dim
    self.w = np.random.normal(size=(input_dim,))
    self.b = 0.

  def __call__(self, x):
    return self.forward(x)
  
  def forward(self, x):
    return sigmoid(np.matmul(x, self.w) + self.b)
  
  def compute_gradients(self, x, t):
    y = self.forward(x)
    delta = y - t
    dw = np.matmul(x.T, delta)
    db = np.matmul(np.ones(x.shape[0]), delta)
    return dw, db

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

if __name__ == '__main__':
  np.random.seed(1234)

  '''
  1. データの準備
  '''
  #XOR
  x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
  t = np.array([0 , 1 , 1 , 0])
  '''
  2. モデルの構築
  '''
  model = LogisticRegression(input_dim=2)
  '''
  3. モデルの学習
  '''
  def compute_loss(t, y):
    return (-t * np.log(y) - (1 - t) * np.log(1 - y)).sum()
  
  def train_step(x, t):
    dw, db = model.compute_gradients(x, t)
    model.w = model.w - 0.1 * dw
    model.b = model.b - 0.1 * db
    loss = compute_loss(t, model(x))
    return loss

  epochs = 100

  for epoch in range(epochs):
    train_loss = train_step(x, t) #パッチ学習

    if epoch % 10 == 0 or epoch == epochs - 1:
      print('epoch: {}, loss: {: .3f}' .format(epoch+1, train_loss))
  '''
  4. モデルの評価
  '''
  for input in x :
    print('{} => {: .3f}'.format(input, model(input)))


epoch: 1, loss:  3.003
epoch: 11, loss:  2.884
epoch: 21, loss:  2.840
epoch: 31, loss:  2.815
epoch: 41, loss:  2.799
epoch: 51, loss:  2.789
epoch: 61, loss:  2.783
epoch: 71, loss:  2.779
epoch: 81, loss:  2.777
epoch: 91, loss:  2.775
epoch: 100, loss:  2.774
[0 0] =>  0.513
[0 1] =>  0.485
[1 0] =>  0.519
[1 1] =>  0.491


In [2]:
import numpy as np

class MLP(object):
  '''
  多層パーセプトロン
  '''
  def __init__(self, input_dim, hidden_dim, output_dim):
    '''
    引数:
          input_dim: 入力層の次元
          hidden_dim: 隠れ層の次元
          output_dim: 出力層の次元
    '''
    self.l1 = Layer(input_dim=input_dim,
                    output_dim=hidden_dim,
                    activation=sigmoid,
                    dactivation=dsigmoid)
    
    self.l２ = Layer(input_dim=hidden_dim,
                    output_dim=output_dim,
                    activation=sigmoid,
                    dactivation=dsigmoid)
    
    self.layers = [self.l1, self.l2]

  def __call__(self, x):
    return self.forward(x)

  def forward(self, x):
    h = self.l1(x)
    y = self.l2(h)
    return y

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def dsigmoid(x):
  return sigmoid(x) * (1 - sigmoid(x))

class Layer(object):
  def __init__(self, input_dim, output_dim, activation, dactivation):
    '''
    インスタンス変数:
      W: 重み
      b: バイアス
      activation: 活性化関数
      dactivation: 活性化関数の微分
    '''
    self.W = np.random.normal(size=(input_dim, output_dim))
    self.b = np.zeros(output_dim)

    self.activation = activation
    self.dactivation = dactivation
  
  def __call__(self, x):
    return self.forward(x)
  
  def forward(self, x):
    self._input = x
    self._pre_activation = np.matmul(x, self.W) + self.b
    return self.activation(self._pre_activation)

  def backward(self, delta, W):
    delta = self.dactivation(self._pre_activation) * np.matmul(delta, W.T)
    return delta

  def compute_gradients(self, delta):
    dW = np.matmul(self._input.T, delta)
    db = np.matmul(np.ones(self._input.shape[0]), delta)
    return dW, db

if __name__ == '__main__':
  np.random.seed(123)

  '''
  1. データの準備
  '''
  #XOR
  x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
  t = np.array([[0], [1], [1], [0]])
  '''
  2. モデルの構築
  '''
  model = MLP(2, 2, 1)
  '''
  3. モデルの学習
  '''
  def compute_loss(t, y):
    return (-t * np.log(y) - (1 - t) * np.log(1 - y)).sum()
  
  def train_step(x, t):
    y = model(x)
    for i, layer in enumerate(model.layers[: : -1]):
      if i == 0:
        delta = y - t
      else:
        delta = layer.backward(delta, W)

      dW, db = layer.compute_gradients(delta)
      layer.W = layer.W - 0.1 * dW
      layer.b = layer.b - 0.1 * db

      W = layer.W
    
    loss = compute_loss(t, y)
    return loss
  
  epochs = 1000

  for epoch in range(epochs):
    train_loss = train_step(x, t)

    if epoch % 100 == 0 or epoch == epochs - 1:
      print('epoch: {}, loss: {:.3f}'
      .format(epoch+1,train_loss))

  '''
  4. モデルの評価
  '''
  for input in x:
    print('{} => {:.3f}'.format(input, model(input)[0]))



epoch: 1, loss: 2.940
epoch: 101, loss: 2.696
epoch: 201, loss: 2.572
epoch: 301, loss: 2.403
epoch: 401, loss: 2.250
epoch: 501, loss: 2.110
epoch: 601, loss: 1.849
epoch: 701, loss: 1.274
epoch: 801, loss: 0.738
epoch: 901, loss: 0.470
epoch: 1000, loss: 0.336
[0 0] => 0.096
[0 1] => 0.909
[1 0] => 0.944
[1 1] => 0.078
